<a href="https://colab.research.google.com/github/ToukirSabugar/ANOMALY-DETECTION/blob/main/ANOMALYDETECTION_OCSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
# Data processing
import pandas as pd
import numpy as np
from collections import Counter
# Visualization
import matplotlib.pyplot as plt
# Model and performance
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
dataset = pd.read_csv('/content/city_dayL.csv')


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29532 entries, 0 to 29531
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29532 non-null  object 
 1   Date        29532 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25950 non-null  float64
 5   NO2         25947 non-null  float64
 6   NOx         25347 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27473 non-null  float64
 9   SO2         25678 non-null  float64
 10  O3          25510 non-null  float64
 11  Benzene     23909 non-null  float64
 12  Toluene     21491 non-null  float64
 13  Xylene      11423 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB


In [ ]:
dataset.isnull().sum()

City              0
Date              0
PM2.5          4599
PM10          11141
NO             3582
NO2            3585
NOx            4185
NH3           10329
CO             2059
SO2            3854
O3             4022
Benzene        5623
Toluene        8041
Xylene        18109
AQI            4682
AQI_Bucket     4682
dtype: int64

In [ ]:
newdf = dataset.fillna(dataset.mean())


<ipython-input-7-8105178ecb2f>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  newdf = dataset.fillna(dataset.mean())


In [ ]:
newdf.head(5)

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,67.450578,118.127103,0.92,18.22,17.15,23.483476,0.92,27.64,133.36,0.00,0.02,0.00,166.463581,NaN
1,Ahmedabad,2015-01-02,67.450578,118.127103,0.97,15.69,16.46,23.483476,0.97,24.55,34.06,3.68,5.50,3.77,166.463581,NaN
2,Ahmedabad,2015-01-02,67.450578,118.127103,0.97,15.69,16.46,23.483476,0.97,24.55,34.06,3.68,5.50,3.77,166.463581,NaN
3,Ahmedabad,2015-01-03,67.450578,118.127103,17.40,19.30,29.70,23.483476,17.40,29.07,30.70,6.80,16.40,2.25,166.463581,NaN
4,Ahmedabad,2015-01-04,67.450578,118.127103,1.70,18.48,17.97,23.483476,1.70,18.59,36.08,4.43,10.14,1.00,166.463581,NaN


In [ ]:
categorical_columns = newdf.select_dtypes(include=['object', 'category']).columns
df_numerical = newdf.drop(columns=categorical_columns)

In [ ]:
df_numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29532 entries, 0 to 29531
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   PM2.5    29532 non-null  float64
 1   PM10     29532 non-null  float64
 2   NO       29532 non-null  float64
 3   NO2      29532 non-null  float64
 4   NOx      29532 non-null  float64
 5   NH3      29532 non-null  float64
 6   CO       29532 non-null  float64
 7   SO2      29532 non-null  float64
 8   O3       29532 non-null  float64
 9   Benzene  29532 non-null  float64
 10  Toluene  29532 non-null  float64
 11  Xylene   29532 non-null  float64
 12  AQI      29532 non-null  float64
dtypes: float64(13)
memory usage: 2.9 MB


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X_scaled = scaler.fit_transform(df_numerical)

In [ ]:
column_names = ["PM2.5", "PM10","NO", "NO2", "NOx", "NH3","CO", "SO2", "O3", "Benzene", "Toluene", "Xylene", "AQI"]


In [ ]:
df_scaled = pd.DataFrame(X_scaled, columns=column_names)


In [ ]:
print(df_scaled.head())

          PM2.5          PM10        NO       NO2       NOx  NH3        CO  \
0 -2.391898e-16 -3.975146e-16 -0.779733 -0.450740 -0.517053  0.0 -0.197833   
1 -2.391898e-16 -3.975146e-16 -0.777392 -0.561026 -0.540589  0.0 -0.190387   
2 -2.391898e-16 -3.975146e-16 -0.777392 -0.561026 -0.540589  0.0 -0.190387   
3 -2.391898e-16 -3.975146e-16 -0.008151 -0.403662 -0.088975  0.0  2.256181   
4 -2.391898e-16 -3.975146e-16 -0.743214 -0.439407 -0.489083  0.0 -0.081684   

        SO2        O3   Benzene   Toluene    Xylene  AQI  
0  0.775204  4.903524 -0.230626 -0.509611 -0.780763  0.0  
1  0.592457 -0.021397  0.028058 -0.187906  0.177965  0.0  
2  0.592457 -0.021397  0.028058 -0.187906  0.177965  0.0  
3  0.859777 -0.188040  0.247376  0.451983 -0.208578  0.0  
4  0.239974  0.078788  0.080778  0.084487 -0.526458  0.0  


In [ ]:
df1 = df_scaled
#OCSVM = OneClassSVM(nu=0.09888888888888889, kernel='sigmoid', gamma=0.05555555555555556, degree=3)
OCSVM = OneClassSVM(nu=0.017272727272727273, kernel='sigmoid', gamma=0.08545454545454546, degree=3)
model=OCSVM.fit(df1)

In [ ]:
anomaly_predictions = model.decision_function(df1)

In [ ]:
anomaly_predictions

array([60.30382642, 82.65518234, 82.65518234, ...,  8.30762948,
        8.73498081, 26.58600832])

In [ ]:
# Define anomaly threshold

anomaly_threshold = 55  # Adjust the threshold as per your requirements
#print("TA17")

In [ ]:
# Determine anomaly predictions based on threshold

anomaly_predictions = np.where(anomaly_predictions < anomaly_threshold, 0, 1)



In [ ]:
mean_score = np.mean(anomaly_predictions)
mean_score


0.6142150887173236

In [ ]:
labels = anomaly_predictions


In [ ]:
from sklearn.metrics import silhouette_score

# Calculate silhouette score
silhouette_avg = silhouette_score(df1, labels)
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.07564767030866058
